<a href="https://colab.research.google.com/github/massivetriceps/AI/blob/main/week5/assignment/diabetes_regression_prc1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving diabetes.csv to diabetes.csv


In [ ]:
df = pd.read_csv('diabetes.csv')

In [ ]:
X = df.drop('Outcome', axis=1)
y = df['Outcome'].values.astype(np.float32)

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test.reshape(-1, 1), dtype=torch.float32)

In [ ]:
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
class DiabetesRegressor(nn.Module):
    def __init__(self):
        super(DiabetesRegressor, self).__init__()
        self.fc1 = nn.Linear(8, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # 회귀: 출력 노드 1개

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)  # sigmoid 없음
        return x

In [ ]:
model = DiabetesRegressor()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 20
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")


Epoch 1/20, Loss: 0.2654
Epoch 2/20, Loss: 0.1929
Epoch 3/20, Loss: 0.1851
Epoch 4/20, Loss: 0.1680
Epoch 5/20, Loss: 0.1627
Epoch 6/20, Loss: 0.1626
Epoch 7/20, Loss: 0.1578
Epoch 8/20, Loss: 0.1510
Epoch 9/20, Loss: 0.1499
Epoch 10/20, Loss: 0.1502
Epoch 11/20, Loss: 0.1446
Epoch 12/20, Loss: 0.1484
Epoch 13/20, Loss: 0.1500
Epoch 14/20, Loss: 0.1413
Epoch 15/20, Loss: 0.1473
Epoch 16/20, Loss: 0.1482
Epoch 17/20, Loss: 0.1396
Epoch 18/20, Loss: 0.1354
Epoch 19/20, Loss: 0.1329
Epoch 20/20, Loss: 0.1377


In [ ]:
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    mse = mean_squared_error(y_test, predictions)
    print(f"Test MSE: {mse:.4f}")

Test MSE: 0.1447
